Build a graph from TO using D3

In [2]:
# import libraries (including our short d3_lib script)
from IPython.core.display import HTML
import d3_lib, random
import re

### get all the TO classes and the sublcassOf relations

In [3]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

#CONSTRUCT 
#{
#    ?x rdfs:subClassOf ?o .
#}
SELECT ?x ?xlabel ?o ?olabel
WHERE
{
    ?x rdfs:subClassOf ?o .
    filter(!isblank(?o)) 
    filter(!regex(str(?o), "PATO"))
    filter(regex(str(?o), "TO"))
    ?x rdfs:label ?xlabel . 
    ?o rdfs:label ?olabel .
}' > to.json

### get for each TO class its level in the hierarchy

will be used to customise the graph

In [4]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

select ?super (count(?mid) as ?distance) { 
  ?super rdfs:subClassOf* ?mid .
  ?mid rdfs:subClassOf+ <http://purl.obolibrary.org/obo/TO_0000387> .
}
group by ?super ?sub 
order by ?super ?sub' > level.json

### get for each TO class its type

In [5]:
%%script bash
arq --results JSON --data https://raw.githubusercontent.com/Planteome/plant-trait-ontology/master/plant-trait-ontology.obo.owl '
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>  
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>  
PREFIX owl: <http://www.w3.org/2002/07/owl#> 
PREFIX oio: <http://www.geneontology.org/formats/oboInOwl#>  
PREFIX obo: <http://purl.obolibrary.org/obo/>  

select ?sub ?mid { 
  ?sub rdfs:subClassOf* ?mid .
  ?mid rdfs:subClassOf <http://purl.obolibrary.org/obo/TO_0000387> .
}
group by ?mid ?sub 
' > type.json

### create the JSON file used by d3js

In [6]:
import json

with open('to.json') as data_file:    
    data = json.load(data_file)
    
with open('level.json') as level_file:    
    level = json.load(level_file)
    
with open('type.json') as type_file:    
    type = json.load(type_file) 
    
    list = data["results"]["bindings"]
    
    graph = {"nodes": [], "links": []}
    
    for entry in list:
        x = 1
        o = 1
        #check if the nodes already exist in the graph
        for e in graph["nodes"]:
            if e["name"] == entry["xlabel"]["value"]:
                x=graph["nodes"].index(e)
            if e["name"] == entry["olabel"]["value"]:
                o=graph["nodes"].index(e)
        # if the term does not exist, create it        
        if x==1:
            for l in level["results"]["bindings"]:
                if l["super"]["value"] == entry["x"]["value"]:
                    graph["nodes"].append( {"name": entry["xlabel"]["value"], "value": 1/int(l["distance"]["value"]), "id":entry["x"]["value"] } )
                    x=len(graph["nodes"])-1
        if o==1:
            for l in level["results"]["bindings"]:
                if l["super"]["value"] == entry["o"]["value"]:
                    graph["nodes"].append( {"name": entry["olabel"]["value"], "value": 1/int(l["distance"]["value"]), "id":entry["x"]["value"]} )
                    o=len(graph["nodes"])-1

        # create the link
        if  entry["x"]["value"] != entry["o"]["value"]:
            graph["links"].append( {"source": x, "target": o, "value": 10} )
    
    # add the types
    for node in graph["nodes"]:
        for t in type["results"]["bindings"]:
            if t["sub"]["value"] == node["id"]:
                node["group"] = re.findall('\d+', t["mid"]["value"])
        #"group": re.findall('\d+', t["mid"]["value"] ),
        
#data["results"]['bindings'][0]['x']['value']


In [7]:
# visualize as force-directed graph in D3
HTML(d3_lib.set_styles(['force_directed_graph']) + 
'<script src="lib/d3/d3.min.js"></script>' + 
      d3_lib.draw_graph('force_directed_graph', {'data': graph}) )

### Extras
#### Save the graph

In [65]:
with open('data.json', 'w') as outfile:
    json.dump(graph, outfile)